# Exercise 1.04: Tic-Tac-Toe Static Evaluation with a Heuristic Function
In this exercise, you will be performing a static evaluation on the tic-tac-toe game using a heuristic function.

1.- Reuse the code from Steps 1–5 of Activity 1.01

In [2]:
from random import choice

wins= [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [0, 3, 6],
    [1, 4, 7],
    [2, 5, 8],
    [0, 4, 8],
    [2, 4, 6]
]

EMPTY_SIGN='-'
IA_SIGN='X'
PLAYER_SIGN='O'

def prtBoard(board):
    cells=board
    print('Fila/Col     1   2   3 ')
    print('\t1    ' + cells[0] + ' | ' + cells[1] + ' | ' + cells[2])
    print('\t    -----------')
    print('\t2    ' + cells[3] + ' | ' + cells[4] + ' | ' + cells[5])
    print('\t    -----------')
    print('\t3    ' + cells[6] + ' | ' + cells[7] + ' | ' + cells[8])

def player_move(board, fila, columna):
    while True:
      index = 3 * (fila - 1) + (columna - 1)
      if board[index] == EMPTY_SIGN:
          return board[:index] + PLAYER_SIGN + board[index+1:]
      else:
        print('\nFila: '+ str(fila) +' Columna: '+ str(columna))
        print('Esta posicion ya esta ocupada, Elige otra celda...\n\n')
        prtBoard(board)
        fila = int(input('Ingresa la Fila: '))
        columna = int(input('Ingresa la Columna: '))
        
def game_won_by(board):
    for idx in wins:
        if board[idx[0]] == board[idx[1]] == board[idx[2]] != EMPTY_SIGN:
            return board[idx[0]]
    return EMPTY_SIGN


def juego_Gato():
    fin= False
    empty_cell_count = 9
    board = EMPTY_SIGN * 9
    print('TIC TAC TOE GAME\n\n')
    nombre = input('Teclea tu Nombre: ')
    print('Comienza el Juego!')
    while empty_cell_count > 0 and not fin:
        if empty_cell_count % 2 == 1:
            print('\n\n\nTurno de la IA\n')
            board = ia_move(board)
        else:
            print('\n\n\nTurno de '+ nombre)
            fila = int(input('Ingresa la Fila: '))
            columna = int(input('Ingresa la Columna: '))
            print('')
            board = player_move(board, fila, columna)
        prtBoard(board)
        fin = game_won_by(board) != EMPTY_SIGN
        empty_cell_count = sum(1 for cell in board if cell == EMPTY_SIGN) 
    print('Fin Del Juego.')

2.- Create a function that takes the board as input and returns $0$ if the cell is empty, and $-1$ if it's not empty

In [3]:
def boardEmpty(board):
  lista= []
  for cell in board:
      if cell == EMPTY_SIGN:
        lista.append(0)
      else:
        lista.append(-1)
    
  return lista




3.- Create a function that takes the utility vector of possible moves, takes three indices inside the utility vector representing a triple, and returns a function.

  > **Hints**  
  > the returned function will expect a `points` parameter and the `utilities` vector as input and will add points to each cell in $(i, j, k)$, as long as the original value of that cell is non-negative $(>=0)$. In other words, we increased the utility of empty cells only.

In [4]:
def generate_add_score(utilities, i, j, k):
  def Fun_points(points):
    utilities[i]+= points if utilities[i]>=0 else 0
    utilities[j]+= points if utilities[j]>=0 else 0
    utilities[k]+= points if utilities[k]>=0 else 0
  return Fun_points
  

4.- Now, create the utility matrix belonging to any board constellation where you will add the `generate_add_score` function defined previously to update the score. You will also implement the rules that we discussed prior to this activity. These rules are as follows:
  * Two AI signs in a row, column, or diagonal, and the third cell is empty: +1000 for the empty cell.
  * The opponent has two signs in a row, column, or diagonal, and the third cell is empty: +100 for the empty cell.
  * One AI sign in a row, column, or diagonal, and the other two cells are empty: +10 for the empty cells.
  * No AI or opponent signs in a row, column, or diagonal: +1 for the empty cells.

In [5]:
def utility_matrix(board):
  utilities= boardEmpty(board)
  for [ i, j, k] in wins:
    add_puntos= generate_add_score(utilities, i, j, k)
    tres = [board[i], board[j], board[k]]
    if tres.count(EMPTY_SIGN) == 1:
      if tres.count(IA_SIGN) == 2:
        add_puntos(1000)
      elif tres.count(PLAYER_SIGN) == 2:
        add_puntos(100)
    elif tres.count(EMPTY_SIGN) == 2 and tres.count(IA_SIGN) == 1:
      add_puntos(10)
    elif tres.count(EMPTY_SIGN) == 3:
      add_puntos(1)
  return utilities

5.- Create a function that selects the move with the highest utility value. If multiple moves have the same utility, the function returns both moves

In [6]:
def highest_utility(board, sign):
    moves = list()
    utilities = utility_matrix(board)

    for idx in range(len(board)):
        if utilities[idx] == max(utilities):
            moves.append(board[:idx] + sign + board[idx+1:])

    return moves

6.- Run the application.

Output:

```
step 0. Moves: 1
step 1. Moves: 1
step 2. Moves: 8
step 3. Moves: 24
step 4. Moves: 144
step 5. Moves: 83
step 6. Moves: 214
step 7. Moves: 148
step 8. Moves: 172
First player wins: 504
Second player wins: 12
Draw 91
Total 607
```

In [7]:
def all_moves_from_board(board, sign):
    move_list = list()

    for index in range(len(board)):
        if board[index] == EMPTY_SIGN:
            move_list.append(board[:index] + sign + board[index + 1:])

    return move_list


def all_moves_from_board_list(board_list, sign):
  moves = []
  get_moves = highest_utility if sign == IA_SIGN else all_moves_from_board
  for board in board_list:
    moves.extend(get_moves(board, sign))
  return moves

def count_possibilities(): 
  lmoves = [EMPTY_SIGN * 9]
  ia_wins = []
  player_wins = []
  for i in range(9):
    print('Paso ' + str(i) + '. Movimientos: ' +  str(len(lmoves)))
    sign = IA_SIGN if i % 2 == 0 else PLAYER_SIGN
    lmoves = all_moves_from_board_list(lmoves, sign)
    
    for b in lmoves:
      won_by = game_won_by(b)
      if won_by == IA_SIGN:
        ia_wins.append(b)
        lmoves.remove(b)
      elif won_by == PLAYER_SIGN:
        player_wins.append(b)
        lmoves.remove(b)
  print('---Victorias---')
  print('IA: ' + str(len(ia_wins)))
  print('Jugador: ' + str(len(player_wins)))
  print('Draw', str(len(lmoves)))
  print('Total', str(len(ia_wins) + len(player_wins) +  len(lmoves)))
  
count_possibilities()

Paso 0. Movimientos: 1
Paso 1. Movimientos: 1
Paso 2. Movimientos: 8
Paso 3. Movimientos: 24
Paso 4. Movimientos: 144
Paso 5. Movimientos: 83
Paso 6. Movimientos: 214
Paso 7. Movimientos: 148
Paso 8. Movimientos: 172
---Victorias---
IA: 504
Jugador: 12
Draw 91
Total 607
